In [4]:
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [5]:
data = pd.read_csv("../data/raw/niftybank.csv")
data.head()

,date,open,high,low,close,volume
0,2017-07-17 09:14:00,24011.40,24011.40,24011.40,24011.40,0
1,2017-07-17 09:15:00,24011.40,24014.40,23974.95,23974.95,0
2,2017-07-17 09:16:00,23975.45,23981.55,23963.15,23978.95,0
3,2017-07-17 09:17:00,23977.25,23995.30,23976.00,23984.60,0
4,2017-07-17 09:18:00,23983.25,23989.95,23983.25,23987.10,0


In [15]:
data["Date"] = data["date"].str.split(" ").str[0]
data["Time"] = data["date"].str.split(" ").str[1].str.split("+").str[0]
data["Date_Time"] = data['Date'] + " " + data["Time"]
data.drop("Date", axis = 1, inplace = True)
data



,date,open,high,low,close,volume,Time,Date_Time
0,2017-07-17 09:14:00,24011.40,24011.40,24011.40,24011.40,0,09:14:00,2017-07-17 09:14:00
1,2017-07-17 09:15:00,24011.40,24014.40,23974.95,23974.95,0,09:15:00,2017-07-17 09:15:00
2,2017-07-17 09:16:00,23975.45,23981.55,23963.15,23978.95,0,09:16:00,2017-07-17 09:16:00
3,2017-07-17 09:17:00,23977.25,23995.30,23976.00,23984.60,0,09:17:00,2017-07-17 09:17:00
4,2017-07-17 09:18:00,23983.25,23989.95,23983.25,23987.10,0,09:18:00,2017-07-17 09:18:00
...,...,...,...,...,...,...,...,...
616014,2024-03-07 15:25:00,47815.65,47815.65,47788.80,47802.45,0,15:25:00,2024-03-07 15:25:00
616015,2024-03-07 15:26:00,47805.65,47805.65,47790.45,47795.80,0,15:26:00,2024-03-07 15:26:00
616016,2024-03-07 15:27:00,47797.10,47802.95,47782.00,47800.35,0,15:27:00,2024-03-07 15:27:00
616017,2024-03-07 15:28:00,47802.70,47806.05,47783.75,47785.70,0,15:28:00,2024-03-07 15:28:00


In [16]:
data["Date_Time"] = pd.to_datetime(data["Date_Time"], format = "%Y-%m-%d %H:%M:%S")
data.index = data['Date_Time']
data.drop(["date"], axis = 1, inplace = True)
data["Next Date_Time"] = data["Date_Time"].shift(-1)
data

,open,high,low,close,volume,Time,Date_Time,Next Date_Time
Date_Time,,,,,,,,
2017-07-17 09:14:00,24011.40,24011.40,24011.40,24011.40,0,09:14:00,2017-07-17 09:14:00,2017-07-17 09:15:00
2017-07-17 09:15:00,24011.40,24014.40,23974.95,23974.95,0,09:15:00,2017-07-17 09:15:00,2017-07-17 09:16:00
2017-07-17 09:16:00,23975.45,23981.55,23963.15,23978.95,0,09:16:00,2017-07-17 09:16:00,2017-07-17 09:17:00
2017-07-17 09:17:00,23977.25,23995.30,23976.00,23984.60,0,09:17:00,2017-07-17 09:17:00,2017-07-17 09:18:00
2017-07-17 09:18:00,23983.25,23989.95,23983.25,23987.10,0,09:18:00,2017-07-17 09:18:00,2017-07-17 09:19:00
...,...,...,...,...,...,...,...,...
2024-03-07 15:25:00,47815.65,47815.65,47788.80,47802.45,0,15:25:00,2024-03-07 15:25:00,2024-03-07 15:26:00
2024-03-07 15:26:00,47805.65,47805.65,47790.45,47795.80,0,15:26:00,2024-03-07 15:26:00,2024-03-07 15:27:00
2024-03-07 15:27:00,47797.10,47802.95,47782.00,47800.35,0,15:27:00,2024-03-07 15:27:00,2024-03-07 15:28:00


In [1]:
def convert_timeframe(data, tf):
    data = data.copy()  # To avoid modifying the original DataFrame
    
    new_data = {
        "Date_Time": [],
        "Open": [],
        "High": [],
        "Low": [],
        "Close": []
    }
    
    step = 0
    for i in data.index:
        step += 1
        if step == 1:
            o = data.loc[i, "open"]
            t = i
            h = data.loc[i, "high"]
            l = data.loc[i, "low"]
        else:
            if data.loc[i, "high"] > h:
                h = data.loc[i, "high"]
            if data.loc[i, "low"] < l:
                l = data.loc[i, "low"]
            if step == tf or i.day != data.loc[i, "Next_Date_Time"].day:
                step = 0
                c = data.loc[i, "close"]
                new_data["Date_Time"].append(t)
                new_data["Open"].append(o)
                new_data["High"].append(h)
                new_data["Low"].append(l)
                new_data["Close"].append(c)
    
    return pd.DataFrame(new_data)


In [2]:
data_1w = convert_timeframe(data, 10080)
data_1w

NameError: name 'data' is not defined

In [36]:
data45 = convert_timeframe(data, 10080)
data45_original = data45.copy()

In [37]:
data45.set_index("Date_Time", inplace = True)

In [38]:
data45

,Open,High,Low,Close
Date_Time,,,,
2017-07-17 09:14:00,24011.40,24064.70,23914.40,24015.70
2017-07-18 09:14:00,23958.15,24143.75,23958.15,24012.05
2017-07-19 09:14:00,24069.20,24175.80,24069.20,24160.60
2017-07-20 09:14:00,24170.65,24294.80,24170.65,24222.45
2017-07-21 09:14:00,24231.40,24282.00,24074.30,24280.65
...,...,...,...,...
2024-03-02 09:15:00,47377.45,47438.60,47237.00,47294.60
2024-03-04 09:15:00,47318.50,47529.60,47191.65,47425.65
2024-03-05 09:15:00,47265.70,47737.85,47196.75,47582.15


In [39]:
data45.to_csv("../data/raw/5EMA_1WBN.csv")

In [14]:
data_1d = pd.read_csv('../data/raw/5EMA_1DBN.csv')
data_1d.index = pd.to_datetime(data_1d['Date_Time'])

In [8]:
data_5m = pd.read_csv('../data/raw/5EMA_5BN.csv')
data_5m.set_index('Date_Time', inplace = True)
data_5m.index = pd.to_datetime(data_5m.index)
data_5m

,Open,High,Low,Close
Date_Time,,,,
2017-07-17 09:14:00,24011.40,24014.40,23963.15,23987.10
2017-07-17 09:19:00,23986.60,23998.25,23954.05,23982.90
2017-07-17 09:24:00,23983.55,23995.20,23967.95,23984.70
2017-07-17 09:29:00,23983.00,23984.85,23952.20,23954.90
2017-07-17 09:34:00,23954.15,23963.40,23921.20,23940.70
...,...,...,...,...
2024-03-07 15:04:00,47892.75,47899.10,47823.40,47871.40
2024-03-07 15:09:00,47872.10,47881.85,47815.75,47836.10
2024-03-07 15:14:00,47838.90,47871.10,47822.05,47832.65


In [41]:
data

,open,high,low,close,volume
date,,,,,
2017-07-17 09:14:00,24011.40,24011.40,24011.40,24011.40,0
2017-07-17 09:15:00,24011.40,24014.40,23974.95,23974.95,0
2017-07-17 09:16:00,23975.45,23981.55,23963.15,23978.95,0
2017-07-17 09:17:00,23977.25,23995.30,23976.00,23984.60,0
2017-07-17 09:18:00,23983.25,23989.95,23983.25,23987.10,0
...,...,...,...,...,...
2024-03-07 15:25:00,47815.65,47815.65,47788.80,47802.45,0
2024-03-07 15:26:00,47805.65,47805.65,47790.45,47795.80,0
2024-03-07 15:27:00,47797.10,47802.95,47782.00,47800.35,0


In [ ]:
data_1d = convert_timeframe(data, '1D')

In [54]:
data_1d

,open,high,low,close
2017-07-17 00:00:00+00:00,23976.85,23980.90,23974.25,23974.95
2017-07-18 00:00:00+00:00,24105.70,24105.70,24105.70,24105.70
2017-07-19 00:00:00+00:00,24128.95,24128.95,24128.95,24128.95
2017-07-20 00:00:00+00:00,24210.55,24210.55,24210.55,24210.55
2017-07-21 00:00:00+00:00,24235.95,24235.95,24235.95,24235.95
...,...,...,...,...
2024-03-03 00:00:00+00:00,NaN,NaN,NaN,NaN
2024-03-04 00:00:00+00:00,47328.35,47330.75,47301.70,47307.25
2024-03-05 00:00:00+00:00,47334.00,47337.50,47299.00,47313.80
2024-03-06 00:00:00+00:00,47748.80,47784.30,47739.50,47782.45


In [16]:
test_1d = pd.read_csv('../data/raw/5EMA_1DBN.csv')
test_1d

,Date_Time,Open,High,Low,Close
0,2017-07-17 09:14:00,24011.40,24064.70,23914.40,24015.70
1,2017-07-18 09:14:00,23958.15,24143.75,23958.15,24012.05
2,2017-07-19 09:14:00,24069.20,24175.80,24069.20,24160.60
3,2017-07-20 09:14:00,24170.65,24294.80,24170.65,24222.45
4,2017-07-21 09:14:00,24231.40,24282.00,24074.30,24280.65
...,...,...,...,...,...
1643,2024-03-02 09:15:00,47377.45,47438.60,47237.00,47294.60
1644,2024-03-04 09:15:00,47318.50,47529.60,47191.65,47425.65
1645,2024-03-05 09:15:00,47265.70,47737.85,47196.75,47582.15
1646,2024-03-06 09:15:00,47451.65,48161.25,47442.25,47959.85


In [52]:
data_1d.iloc[1]

open     24105.7
high     24105.7
low      24105.7
close    24105.7
Name: 2017-07-18 00:00:00+00:00, dtype: float64

In [53]:
test_1d.iloc[1]

Date_Time    2017-07-18 09:14:00
Open                    23958.15
High                    24143.75
Low                     23958.15
Close                   24012.05
Name: 1, dtype: object

In [13]:
data_1w = convert_timeframe(data_5m, 'w')

In [14]:
data_1w

,Open,High,Low,Close
Date_Time,,,,
2017-07-23,24011.40,24307.70,23914.40,24274.70
2017-07-30,24240.60,25025.90,24240.60,24806.90
2017-08-06,24880.60,25190.40,24599.15,24823.95
2017-08-13,24854.15,24974.05,23885.35,24101.10
2017-08-20,24058.15,24495.20,23936.95,24140.70
...,...,...,...,...
2024-02-11,45688.25,46181.20,44859.15,45663.80
2024-02-18,45673.60,46693.40,44633.85,46505.70
2024-02-25,46503.20,47363.40,46317.70,46724.95
